# Porfolio Heap Map

## Setup

In [ ]:
import yfinance as yf
import matplotlib.pyplot as plt
import ipywidgets as widgets
import pandas as pd
import numpy as np

In [ ]:
def log_return_DF(tick:str,period_t:str) -> pd.DataFrame:
    """
    
    Parameters
    ----------
    tick : str
        Ticker symbol
    period_t : str
        Valid periods: [1d, 5d, 1mo, 3mo, 6mo, 1y, 2y, 5y, 10y, ytd, max]

    Returns
    -------
    pd.DataFrame:
        The DataFrame contains the Log Returns and the Day

    Description
    -----------
    Fuction that takes the parameters: stock ticker, the period of consideration.
    And returns a DataFrame with the log returns

    Examples
    --------
    >>> log_return_DF("TSLA","1mo")
    """

    tik = yf.Ticker(tick)
    Raw_data = tik.history(period=period_t)
    Raw_data[f"Returns {tick}"] = np.log(Raw_data['Close'] / Raw_data['Close'].shift(1))
    Raw_data.reset_index(inplace=True)
    Raw_data['Day'] = Raw_data['Date'].apply(lambda x: x.date())
    Data_returns= Raw_data[['Day',f"Returns {tick}"]]
    Data_returns=Data_returns[1:]
    
    return Data_returns



def logs_of_stocks(list_tickers: list,period:str) -> tuple:

    """
    
    Parameters
    ----------
    list_tickers : list
        List of ticker symbols representing the portfolio.
    period : str
        Valid periods: [1d, 5d, 1mo, 3mo, 6mo, 1y, 2y, 5y, 10y, ytd, max].

    Returns
    -------
    tuple:
        (list_dataframes,list_lens)
        [0] List of DataFrames having the Log Returns
        [1] Lengths of the DataFrames


    Description
    -----------
    Fuction that takes the parameters: list of stock ticker strings and the period of consideration.
    Returns a tuple with a list of DataFrames and a list of the lenthgs of this DataFrames 

    Examples
    --------
    >>> logs_of_stocks(["TSLA","GM"],"1mo")
    """

    list_dataframes=[]
    lens=[]
    
    for name in list_tickers:
        single_log=log_return_DF(name,period)
        list_dataframes.append(single_log)
        lens.append(len(single_log))

    return list_dataframes,lens



def merger(return_tuple: tuple) -> pd.DataFrame:

    """
    
    Parameters
    ----------
    return_tuple : tuple
        Tuple from the logs_of_stocks() function

    Returns
    -------
    returns_DF: DataFrame
        A DataFrame with all the returns of the portfolio.


    Description
    -----------
    Fuction that takes as parameters the tuple from logs_of_stocks() function and mergers (inner)
    all the DataFrames with the returns of the

    """

    lens=return_tuple[1]
    logs=return_tuple[0]
    index_long=lens.index(max(lens))
    logs[0], logs[index_long] = logs[index_long],logs[0]
    returns_DF=logs[0]

    for dfr in logs[1:]:
        returns_DF = pd.merge(returns_DF,dfr,on="Day",how="inner")
    
    return returns_DF

In [ ]:
period=widgets.Dropdown(
    options=["1d","5d","1mo","3mo","6mo","1y","2y","5y","10y","ytd","max"],
    value="ytd",
    description='Period:',
)
List_Tickers = widgets.TagsInput(
    value=['tsla', "isp.mi","SPY","^VIX","GC=F"],
    allow_duplicates=False
)

## Interaction

In [ ]:
widgets.VBox([List_Tickers,period])

## Calculations

In [ ]:
pot_log=logs_of_stocks(List_Tickers.value,period.value)
raw_ret=merger(pot_log)
clean_ret=raw_ret.reset_index(drop=True)
dataframe_returns=clean_ret.corr()

## Plot

In [ ]:
%matplotlib widget

plt.imshow(dataframe_returns, cmap='cool', interpolation='nearest')
plt.xticks(np.arange(len(dataframe_returns.columns)), dataframe_returns.columns,rotation='vertical')
plt.yticks(np.arange(len(dataframe_returns.columns)), dataframe_returns.columns)

plt.gca().invert_xaxis()
plt.colorbar()
plt.title("Portfolio Return Correlations")

plt.show()